In [ ]:
! nvidia-smi

In [ ]:
! pip install huggingface_hub
! pip install transformers sentencepiece

In [3]:
# ! huggingface-cli login
# presnet setting has bugs?

In [6]:
from transformers import AutoModelForCausalLM as atcl
import torch

OSError: [WinError 182] 操作系统无法运行 %1。 Error loading "d:\Acer App\Anaconda\Anaconda3\lib\site-packages\torch\lib\shm.dll" or one of its dependencies.

In [ ]:
# test_docs=[text.page_content for text in docs[:11]]
query="promopt under construction."#risk_template(test_docs[0])
inputs_ids=tokenizer(query, return_tensors="pt").input_ids.to("cuda")

In [ ]:
generate_result=model.generate(inputs_ids, max_length=1000)
print(tokenizer.decode(generate_result[0], skip_special_tokens=True))

# this is another embedding method, dont know whether comptable with current docs format
# embed_tokenizer=AutoTokenizer.from_pretrained("sentence-transformers/all-MiniLM-L6-v2")
# embed_model=AutoModel.from_pretrained("sentence-transformers/all-MiniLM-L6-v2")

In [ ]:
model_trans="sentence-transformers/all-MiniLM-L6-v2"
model_kwargs={"device": 0}
embedding=HuggingFaceEmbeddings(model_name=model_trans, model_kwargs=model_kwargs)

In [ ]:
prefix_llm=transformers.pipeline(
    model=model, tokenizer=tokenizer,
    return_full_text=True,
    task="text-generation",# temperature=0,
    top_p=0.15, top_k=15,
    max_new_tokens=1060, repetition_penalty=1.2, do_sample=True
)
llm=HuggingFacePipeline(pipeline=prefix_llm, cache=False) # , model_kwargs={'temperature':0}

### LangChiain Output

In [ ]:
import asyncio
# Also pick keywords to label what ENTIRE sentence talked about from given tags: [environment science, environment studies, engineering civil, engineering industrial, management, engineering manufacturing, constructing building, constructing technology, transportation, finance]
# keyword and tags
instruction ="""help me to identify and summarize risks in the given sentence "{words}"? please mark risk out also list out the type.
    If you cannot judge the organizations or there is no matched tag to label the entire sentence, please return 'None'. Show risks, type of risks appeared in sentence separately.

    Answer example: "
    risks: what risk \n

    risk_type : \n
    Attention, you can only predict the output in above given criteria without any explanation and no more than 40 words.
    """
system_prompt = "You are an expert and summarization and expressing key ideas succintly"

template = get_prompt(instruction, system_prompt)
print(template)
prompt = PromptTemplate(template=template, input_variables=["words"])
llm_chain = LLMChain(prompt=prompt, llm=llm)

kwarky_list=[]
testing_dict=[val.page_content for val in docs[:6]]
text=docs[142].page_content
output = llm_chain.run(text)
print(output)

In [ ]:
async def async_generate(achain, context):
    resp = await achain.acall(context)
    print(resp)
    return resp

async def resp_feedback():
  tasks = [async_generate(llm_chain, context) for context in testing_dict]
  await asyncio.gather(*tasks)

await resp_feedback()

#### Load_qa_chain method to output

In [ ]:
chain=load_qa_chain(llm=llm, chain_type="stuff")
res_test=chain.run({"words": docs[4].page_content,
                    "question": query,
                    "input_documents": [docs[4]]})
total_len=sum([len(docs[i].page_content) for i in range(len(docs))])
print(res_test)

#### LLMChain to output

In [ ]:
schema={
    "properties": {
        "risks": {
            "type": "string",
            "description": "any unexpected event that can affect your project. e.g.,Construction cost overrun, Ship collision and grounding"
        },
        "risk_type": {
            "type": "string",
            "description": "the area risk belong to. e.g. Financial, Technical "
        },
        "keywords":{
            "type": "string"
        },
        "tags": {
            "type": "string",
            "description": "select most realted words from given list: [environment science, environment studies, engineering civil, engineering industrial, management, engineering manufacturing, constructing building, constructing technology, transportation, finance]"
        }
    },
    "required": ["risks", "keywords", "tags"]
}
sum_chain=create_extraction_chain(schema, llm, prompt=query)

In [ ]:
sum_chain.run("this is an emergence project")